In [157]:
import os
import polars as pl

In [196]:
kanaly_chceme = ['ČT1','ČT2','Prima','NOVA']
df = pl.read_parquet(os.path.join("data","fdb.parquet")).filter(pl.col("kanal").is_in(kanaly_chceme))

In [194]:
df.group_by("kanal").len()

kanal,len
cat,u32
"""NOVA""",159461
"""Prima""",163569
"""ČT1""",239798
"""ČT2""",248327


In [159]:
ruzne_roky = df.filter(pl.col('kanal').is_in(kanaly_chceme)).group_by('nazev').agg(pl.col('datum').dt.year().unique().len()).filter(pl.col('datum') >= 10).select(pl.col('nazev')).to_series().to_list()

In [160]:
df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
).filter(
    pl.col('nazev').str.contains('Pelíš')
)

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Pelíšky""",14,8,1.75


In [161]:
df.filter(pl.col("nazev") == "Pelíšky")

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2024-08-20 20:10:00,"""Pelíšky""",null,"""Komedie ČR (1999). Jedna z nej…","""film/pelisky/25517"""
"""ČT1""",2020-12-24 21:55:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2016-12-24 21:45:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2019-06-09 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2013-12-24 20:30:00,"""Pelíšky""",null,"""Film ČR (1999). Jedna z nejobl…","""film/pelisky/25517"""
"""ČT1""",2019-12-24 22:05:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""
"""ČT1""",2015-08-02 20:00:00,"""Pelíšky""",null,"""Komed. drama ČR (1999). Jedna …","""film/pelisky/25517"""
"""ČT1""",2023-12-24 22:00:00,"""Pelíšky""",null,"""Komedie ČR (1999). Jedna z nej…","""film/pelisky/25517"""
"""ČT1""",2016-07-17 20:00:00,"""Pelíšky""",null,"""Komediální drama ČR (1999). Je…","""film/pelisky/25517"""


In [162]:
df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
).head(150)

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Novoroční přípitek""",37,0,inf
"""Vánoční posezení u muziky""",24,0,inf
"""Silvestrovské Televizní noviny""",18,0,inf
"""Dr. Dolittle""",17,0,inf
"""Veselé Vánoce přejí chobotnice""",17,0,inf
"""U muziky na Silvestra""",17,0,inf
"""Silvestry Vladimíra Menšíka""",16,0,inf
"""Dr. Dolittle 2""",16,0,inf
"""Mississippi Queen z Pohořelic""",14,0,inf


In [163]:
vyber_predbezny = df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
)

In [164]:
vyber_predbezny.filter(pl.col("nazev").str.contains("Četník"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64


In [165]:
vyber_predbezny.filter(pl.col("nazev").str.contains("tanec"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64


In [166]:
vyber_predbezny.filter(pl.col("nazev").str.contains("Světáci"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Světáci""",21,67,0.313433


In [167]:
vyber_predbezny.filter(pl.col("nazev").str.contains("Smrtonos"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64


In [168]:
df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).sort(
    by=['pomer','prosinec'], descending=True
).filter(
    pl.col("nazev").str.contains("S tebou mě")
)

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""S tebou mě baví svět""",22,20,1.1


In [169]:
df.filter(pl.col("nazev").str.contains("My Fair"))

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT2""",2008-12-28 15:10:00,"""My Fair Lady (?)""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2022-12-25 13:25:00,"""My Fair Lady""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT1""",2007-12-23 22:05:00,"""My Fair Lady (?)""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2009-12-25 12:00:00,"""My Fair Lady (1,2/2)""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2012-12-30 17:20:00,"""My Fair Lady""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2022-12-28 09:30:00,"""My Fair Lady""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2012-12-29 16:40:00,"""My Fair Lady""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2022-01-01 15:10:00,"""My Fair Lady""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""
"""ČT2""",2014-01-04 15:45:00,"""My Fair Lady""","""21.12.2024 v 20:15 (ČT :D / ČT…","""Další díl/opakování:21.12.2024…","""film/my-fair-lady/13262"""


In [170]:
df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).sort(
    by=['pomer','prosinec'], descending=True
).filter(
    pl.col("nazev").str.contains("ho")
)

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Veselé Vánoce přejí chobotnice""",17,0,inf
"""Mississippi Queen z Pohořelic""",14,0,inf
"""Silvestr svobodného pána""",13,0,inf
"""Někdo to rád horké""",14,1,14.0
"""Dunston: Sám v hotelu""",17,3,5.666667
"""Hana a Petr Ulrychovi - Javoro…",11,4,2.75
"""Chobotnice z II. patra""",32,14,2.285714
"""Bohouš""",15,7,2.142857
"""Dařbuján a Pandrhola""",23,16,1.4375


In [182]:
vyber_vypocitany_sirsi = df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).sort(
    by=['pomer','prosinec'], descending=True
)

In [192]:
df.filter(pl.col("nazev").str.contains("Smrtonosná past")).sort(by="datum")

kanal,datum,nazev,extra,noticka,url
cat,datetime[μs],str,str,str,str
"""ČT1""",2007-06-02 21:00:00,"""Smrtonosná past (?)""","""25.11.2024 v 00:55 (JOJ)27.11.…","""Další díl/opakování:25.11.2024…","""film/smrtonosna-past-die-hard/…"
"""ČT1""",2007-06-09 21:00:00,"""Smrtonosná past 2 (?)""",null,null,"""film/smrtonosna-past-2-die-har…"
"""ČT1""",2007-06-16 21:10:00,"""Smrtonosná past 3 (?)""","""03.01.2025 v 23:30 (Markíza)""","""Další díl/opakování:03.01.2025…","""film/smrtonosna-past-3-die-har…"
"""ČT2""",2008-01-11 22:15:00,"""Film o filmu: Smrtonosná past…",null,null,null
"""ČT1""",2008-01-19 04:15:00,"""Film o filmu: Smrtonosná past…",null,null,null
"""ČT2""",2008-04-03 01:35:00,"""Film o filmu: Smrtonosná past…",null,null,null
"""ČT1""",2008-04-12 21:20:00,"""Smrtonosná past (?)""","""25.11.2024 v 00:55 (JOJ)27.11.…","""Další díl/opakování:25.11.2024…","""film/smrtonosna-past-die-hard/…"
"""ČT1""",2008-04-19 21:10:00,"""Smrtonosná past 2 (?)""",null,null,"""film/smrtonosna-past-2-die-har…"
"""ČT2""",2008-04-26 19:20:00,"""Film o filmu: Smrtonosná past…",null,null,null


In [186]:
vyber_vypocitany_sirsi.filter(pl.col("nazev").str.contains("Smrtonosná"))

nazev,prosinec,unor-listopad,pomer
str,u32,u32,f64
"""Smrtonosná zbraň 4""",4,11,0.363636


In [171]:
vyber_vypocitany = df.filter(
    pl.col('nazev').is_in(ruzne_roky)
).group_by(
    ['nazev',pl.col('datum').dt.ordinal_day()]
).len(
).filter(
    (pl.col('datum') <= 1) | (pl.col('datum') >= 357)
).group_by('nazev').agg(pl.col('len').sum()).rename({'len':'prosinec'}).join(
    df.filter(
        pl.col('nazev').is_in(ruzne_roky)
    ).group_by(
        ['nazev',pl.col('datum').dt.month()]
    ).len(
    ).filter(
        pl.col('datum').is_between(2,11)
    ).group_by('nazev').agg(
        pl.col('len').sum()
    ).rename({'len':'unor-listopad'}),
    how='left',
    on='nazev'
).fill_null(0).with_columns(
    (pl.col('prosinec') / pl.col('unor-listopad')).alias('pomer')
).filter(
    (pl.col('prosinec') <= 60) & (pl.col('pomer') > 1)
).filter(
    pl.col('prosinec') >= 10
).sort(
    by=['pomer','prosinec'], descending=True
)

In [172]:
print(vyber_vypocitany.drop("unor-listopad"))

shape: (94, 3)
┌─────────────────────────────────┬──────────┬──────────┐
│ nazev                           ┆ prosinec ┆ pomer    │
│ ---                             ┆ ---      ┆ ---      │
│ str                             ┆ u32      ┆ f64      │
╞═════════════════════════════════╪══════════╪══════════╡
│ Novoroční přípitek              ┆ 37       ┆ inf      │
│ Vánoční posezení u muziky       ┆ 24       ┆ inf      │
│ Silvestrovské Televizní noviny  ┆ 18       ┆ inf      │
│ Veselé Vánoce přejí chobotnice  ┆ 17       ┆ inf      │
│ U muziky na Silvestra           ┆ 17       ┆ inf      │
│ Dr. Dolittle                    ┆ 17       ┆ inf      │
│ Silvestry Vladimíra Menšíka     ┆ 16       ┆ inf      │
│ Dr. Dolittle 2                  ┆ 16       ┆ inf      │
│ Anděl Páně                      ┆ 14       ┆ inf      │
│ Mississippi Queen z Pohořelic   ┆ 14       ┆ inf      │
│ Vánoce s Hutkou                 ┆ 14       ┆ inf      │
│ Čepice                          ┆ 14       ┆ inf      │

In [173]:
vyber_vypocitany_seznam = vyber_vypocitany.select(
    pl.col("nazev")
).to_series(
).to_list(
)

In [174]:
vyber_vypocitany_seznam

['Novoroční přípitek',
 'Vánoční posezení u muziky',
 'Silvestrovské Televizní noviny',
 'Veselé Vánoce přejí chobotnice',
 'U muziky na Silvestra',
 'Dr. Dolittle',
 'Silvestry Vladimíra Menšíka',
 'Dr. Dolittle 2',
 'Anděl Páně',
 'Mississippi Queen z Pohořelic',
 'Vánoce s Hutkou',
 'Čepice',
 'Když čerti nosí štěstí',
 'Silvestr svobodného pána',
 'Prázdniny v Římě',
 'Ráno budeme moudřejší',
 'Štědrý večer pana rady Vacátka',
 'To vánoční šturmování aneb Pokoj lidem dobré vůle',
 'Ten vánoční čas',
 'Vánoce',
 'Do roka a do dna',
 'Silvestr s četníky',
 'Grinch',
 'Poklad',
 'Tajemství mořské panny',
 'Česká mše vánoční v Římě',
 'Z pekla štěstí 2',
 'Rolničky, kam se podíváš',
 'Mrazík',
 'Někdo to rád horké',
 'Tři oříšky pro Popelku',
 'O kominickém učni a dceři cukráře',
 'Vínečko, ne voda',
 'Sám doma 2: Ztracen v New Yorku',
 'Sám doma',
 'Tři bratři',
 'Láska nebeská',
 'Novoroční koncert Vídeňských filharmonik...',
 'Princové jsou na draka',
 'Shrek',
 'Ať přiletí čáp, krá

In [175]:
vyber_vypocitany_seznam.sort()

In [176]:
vyber_vypocitany_seznam

['Adéla ještě nevečeřela',
 'Anděl Páně',
 'Anton Špelec, ostrostřelec',
 'Ať přiletí čáp, královno!',
 'Bohouš',
 'Byl jednou jeden král...',
 'Cesta do Ameriky',
 'Chobotnice z II. patra',
 'Co takhle svatba, princi?',
 'Císař a tambor',
 'Císařův pekař',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Dařbuján a Pandrhola',
 'Do roka a do dna',
 'Dobrý voják Švejk',
 'Dr. Dolittle',
 'Dr. Dolittle 2',
 'Duch nad zlato',
 'Ducháček to zařídí',
 'Dunston: Sám v hotelu',
 'Falešná kočička',
 'Grinch',
 'Hana a Petr Ulrychovi - Javorová píseň',
 'Jak se budí princezny',
 'Katakomby',
 'Když Burian prášil',
 'Když čerti nosí štěstí',
 'Kotva u přívozu',
 'Král Králů',
 'Láska nebeská',
 'Láska rohatá',
 'Mississippi Queen z Pohořelic',
 'Mrazík',
 'Muž se železnou maskou',
 'Na samotě u lesa',
 'Nebojsa',
 'Nejkrásnější hádanka',
 'Nesmrtelná teta',
 'Novoroční koncert Vídeňských filharmonik...',
 'Novoroční přípitek',
 'Někdo to rád horké',
 'O kominickém učni a dceři cukráře',
 'O me

In [177]:
vanocni_vyber = [
    "Prázdniny v Římě",
    "Anděl Páně",
    "Silvestr svobodného pána",
    "Šťastný smolař",
    "Dr. Dolittle 2",
    "Někdo to rád horké",
    "Mrazík",
    "Ať přiletí čáp, královno!",
    "Co takhle svatba, princi?",
    "Císařův pekař",
    "Kotva u přívozu",
    "Cesta do Rokycan",
    "Z pekla štěstí 2",
    "Byl jednou jeden král...",
    "Tři oříšky pro Popelku",
    "Dr. Dolittle",
    "Shrek 2",
    "Pekařův císař",
    "Láska nebeská",
    'Dalskabáty, hříšná ves aneb Zapomenutý čert',
    'Jak se budí princezny',
    'Sám doma 2: Ztracen v New Yorku',
    'Sám doma',
    'Princové jsou na draka',
    'Bohouš',
    'Nebožtíci přejí lásce',
    'Honza málem králem',
    'S čerty nejsou žerty',
    'O medvědu Ondřejovi',
    'Princezna ze mlejna',
    'Kráska a zvíře',
    'Hrátky s čertem',
    'Z pekla štěstí',
    'S tebou mě baví svět',
    'Princ a Večernice',
    'Dunston: Sám v hotelu',
    'Bejvalek se nezbavíš',
    'Veselé Vánoce přejí chobotnice',
    'Shrek',
    'Láska rohatá',
    'Grinch',
    'Světáci'
]

In [178]:
vanocni_vyber.sort()

In [179]:
vanocni_vyber

['Anděl Páně',
 'Ať přiletí čáp, královno!',
 'Bejvalek se nezbavíš',
 'Bohouš',
 'Byl jednou jeden král...',
 'Cesta do Rokycan',
 'Co takhle svatba, princi?',
 'Císařův pekař',
 'Dalskabáty, hříšná ves aneb Zapomenutý čert',
 'Dr. Dolittle',
 'Dr. Dolittle 2',
 'Dunston: Sám v hotelu',
 'Grinch',
 'Honza málem králem',
 'Hrátky s čertem',
 'Jak se budí princezny',
 'Kotva u přívozu',
 'Kráska a zvíře',
 'Láska nebeská',
 'Láska rohatá',
 'Mrazík',
 'Nebožtíci přejí lásce',
 'Někdo to rád horké',
 'O medvědu Ondřejovi',
 'Pekařův císař',
 'Princ a Večernice',
 'Princezna ze mlejna',
 'Princové jsou na draka',
 'Prázdniny v Římě',
 'S tebou mě baví svět',
 'S čerty nejsou žerty',
 'Shrek',
 'Shrek 2',
 'Silvestr svobodného pána',
 'Světáci',
 'Sám doma',
 'Sám doma 2: Ztracen v New Yorku',
 'Tři oříšky pro Popelku',
 'Veselé Vánoce přejí chobotnice',
 'Z pekla štěstí',
 'Z pekla štěstí 2',
 'Šťastný smolař']

In [180]:
vyber_vypocitany_seznam.sort()

In [181]:
vyber_vypocitany_seznam = [x for x in vyber_vypocitany if x != "Kráska a zvíře"]

TypeError: the truth value of a Series is ambiguous

Here are some things you might want to try:
- instead of `if s`, use `if not s.is_empty()`
- instead of `s1 and s2`, use `s1 & s2`
- instead of `s1 or s2`, use `s1 | s2`
- instead of `s in [y, z]`, use `s.is_in([y, z])`


In [ ]:
with open("data_raw/vanocni_vyber_vypocitany.json", "w+", encoding="utf-8") as exp:
    exp.write(json.dumps(vyber_vypocitany_seznam))

In [ ]:
len(vanocni_vyber)

Co do dat (přes Wikidata q):
- země původu
- kino x televizní
- rok výroby
- nejčastější stanice?
- žánr?
- hrají
- scénář

In [ ]:
df.filter(pl.col("nazev").str.contains("Otázky Václava Moravce")).sample(5)

In [ ]:
df.filter(pl.col("nazev").str.contains("Tři oříšky pro Popelku"))